In [2]:
import pandas as pd
from numba import jit,cuda
@jit
def read_data():
    test_csv = pd.read_csv('datasets/cleaned_test_data.csv')
    train_csv = pd.read_csv('datasets/cleaned_trained_data.csv')
    test_csv = test_csv.drop(test_csv.index[0])
    train_csv = train_csv.drop(train_csv.index[0])
    return train_csv,test_csv
train_csv,test_csv = read_data()

C:\Users\Jerry\AppData\Local\Temp\ipykernel_16156\1055007360.py:4: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def read_data():
C:\Users\Jerry\AppData\Local\Temp\ipykernel_16156\1055007360.py:3: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "read_data" failed type inference due to: Unknown attribute 'read_csv' of type Module(<module 'pandas' from 'c:\\Users\\Jerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\pandas\\__init__.py'>)

File "..\..\..\..\..\AppData\Local\Temp\ipykernel_16156\1055007360.py", line 5:
<source missing, REPL/exec in use?>

During: typing of get attribute at C:\Users\Jerry\Ap

In [3]:
categories = {'Society & Culture' :1,'Science & Mathematics':2,'Health':3, 'Education & Reference':4,
            'Computers & Internet' :5,'Sports' :6,'Business & Finance' :7,'Entertainment & Music' : 8,
            'Family & Relationships':9, 'Politics & Government':10}
train_csv['class']= train_csv['class'].map(categories)
train_csv

,text,class
1,n't optical mouse work glass table ? even surf...,5
2,best off-road motorcycle trail ? long-distance...,6
3,Trans Fat ? reduce ? heard tras fat bad body ....,3
4,many planes Fedex ? heard largest airline worl...,7
5,"san francisco bay area , make sense rent buy ?...",7
...,...,...
768321,"believe hopelessness ? believe religion , lack...",1
768322,get horse 's skeletal muscular systems web pri...,4
768323,quest promote racial equality government/media...,10
768324,Ways sell video games ? Like want sell video g...,7


In [25]:
df_balanced = train_csv[train_csv['class']==1].sample(50000)
for index in range(2,11):
    df_balanced = pd.concat([df_balanced,train_csv[train_csv['class']==index].sample(50000)])

In [26]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
import gensim
from gensim.models.doc2vec import TaggedDocument
import re

def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the post.
    """
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(TaggedDocument(v.split(), [label]))
    return labeled


In [27]:
X_train = label_sentences(df_balanced['text'], 'Train')
X_test = label_sentences(test_csv['text'], 'Test')
all_data = X_train + X_test

In [29]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in all_data])

@jit
def tuning():
    for epoch in tqdm(range(30)):
        model_dbow.train(utils.shuffle([x for x in all_data]), total_examples=len(all_data), epochs=1)
        model_dbow.alpha -= 0.002
        model_dbow.min_alpha = model_dbow.alpha
tuning()

C:\Users\Jerry\AppData\Local\Temp\ipykernel_16156\1958504791.py:5: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def tuning():
C:\Users\Jerry\AppData\Local\Temp\ipykernel_16156\1958504791.py:4: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "tuning" failed type inference due to: Untyped global name 'tqdm': Cannot determine Numba type of <class 'type'>

File "..\..\..\..\..\AppData\Local\Temp\ipykernel_16156\1958504791.py", line 6:
<source missing, REPL/exec in use?>

  @jit
C:\Users\Jerry\AppData\Local\Temp\ipykernel_16156\1958504791.py:4: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabl

In [35]:
import numpy as np
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors
    
train_vectors_dbow = get_vectors(model_dbow, len(X_train), 300, 'Train')
test_vectors_dbow = get_vectors(model_dbow, len(X_test), 300, 'Test')

C:\Users\Jerry\AppData\Local\Temp\ipykernel_16156\2789470808.py:14: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  vectors[i] = model.docvecs[prefix]


In [37]:
y_train = df_balanced['class']
y_test = test_csv['class']

In [38]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(train_vectors_dbow, y_train)
logreg = logreg.fit(train_vectors_dbow, y_train)
y_pred = logreg.predict(test_vectors_dbow)

c:\Users\Jerry\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Jerry\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

In [39]:
print('accuracy %s' % accuracy_score(y_pred, y_test.to_numpy().astype(int)))
print(classification_report(y_test.to_numpy().astype(int), y_pred))

accuracy 0.691493889463124
              precision    recall  f1-score   support

           1       0.61      0.54      0.58      3654
           2       0.68      0.72      0.70      2845
           3       0.76      0.77      0.77      3435
           4       0.54      0.49      0.51      3131
           5       0.83      0.85      0.84      3354
           6       0.77      0.83      0.80      2732
           7       0.55      0.53      0.54      2810
           8       0.65      0.67      0.66      3470
           9       0.74      0.75      0.74      3958
          10       0.73      0.75      0.74      3505

    accuracy                           0.69     32894
   macro avg       0.69      0.69      0.69     32894
weighted avg       0.69      0.69      0.69     32894



In [3]:
import numpy as np
np.logspace(-4,4,20)

array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04])